In [1]:
import matplotlib.pyplot as plt
import pandas as pd
import numpy as np
import seaborn  as sns
from sklearn.preprocessing import StandardScaler

## data loading

In [2]:
train_data=pd.read_csv("train.csv")
test_data=pd.read_csv("test.csv")

In [3]:
train_data.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 101763 entries, 0 to 101762
Data columns (total 23 columns):
 #   Column             Non-Null Count   Dtype  
---  ------             --------------   -----  
 0   id                 101763 non-null  int64  
 1   loc                101763 non-null  float64
 2   v(g)               101763 non-null  float64
 3   ev(g)              101763 non-null  float64
 4   iv(g)              101763 non-null  float64
 5   n                  101763 non-null  float64
 6   v                  101763 non-null  float64
 7   l                  101763 non-null  float64
 8   d                  101763 non-null  float64
 9   i                  101763 non-null  float64
 10  e                  101763 non-null  float64
 11  b                  101763 non-null  float64
 12  t                  101763 non-null  float64
 13  lOCode             101763 non-null  int64  
 14  lOComment          101763 non-null  int64  
 15  lOBlank            101763 non-null  int64  
 16  lo

In [4]:
test_data.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 67842 entries, 0 to 67841
Data columns (total 22 columns):
 #   Column             Non-Null Count  Dtype  
---  ------             --------------  -----  
 0   id                 67842 non-null  int64  
 1   loc                67842 non-null  float64
 2   v(g)               67842 non-null  float64
 3   ev(g)              67842 non-null  float64
 4   iv(g)              67842 non-null  float64
 5   n                  67842 non-null  float64
 6   v                  67842 non-null  float64
 7   l                  67842 non-null  float64
 8   d                  67842 non-null  float64
 9   i                  67842 non-null  float64
 10  e                  67842 non-null  float64
 11  b                  67842 non-null  float64
 12  t                  67842 non-null  float64
 13  lOCode             67842 non-null  int64  
 14  lOComment          67842 non-null  int64  
 15  lOBlank            67842 non-null  int64  
 16  locCodeAndComment  678

In [5]:
## data Cleaning

In [6]:
#check null in train data
train_data_null_num=train_data.isnull().sum().sum()
test_data_null_num=test_data.isnull().sum().sum()
print(f"number of null with train {train_data_null_num} , and with test { test_data_null_num}")

number of null with train 0 , and with test 0


In [7]:
#check duplcated in train data
train_data_dup=train_data.duplicated().sum()
test_data_dup=test_data.duplicated().sum()
print(f"number of null with train {train_data_dup} , and with test { test_data_dup}")
#### data is very clean


number of null with train 0 , and with test 0


## draw the data

In [8]:
#  I will  use pca to move X_train to 2 d x1 and x2
data_columns=np.array(train_data.columns)

In [9]:
X_train=np.array(train_data.drop(columns=["defects","id"]))
X_test=np.array(test_data.drop(columns=["id"]))
y_train=np.array(train_data["defects"])


In [10]:
#scale data
std_scaled=StandardScaler()
X_train=std_scaled.fit_transform(X_train)
X_test=std_scaled.fit_transform(X_test)


In [11]:
# df=pd.concat([train_data.drop(columns=["defects","id"]),train_data["defects"]],axis=1)
# sns.pairplot(df, hue='defects', plot_kws={'alpha': 0.5})
# plt.show()


In [12]:
## The model
from sklearn.linear_model import LogisticRegression
lg_model=LogisticRegression(penalty="l2",C=.2,solver="saga")
lg_model.fit(X_train,y_train)
from sklearn.metrics import roc_auc_score

roc_auc_score(y_train,lg_model.predict_proba(X_train)[:,1])

C:\Users\Abdulrahman.Elsadiq\AppData\Local\Programs\Python\Python311\Lib\site-packages\sklearn\linear_model\_sag.py:350: ConvergenceWarning: The max_iter was reached which means the coef_ did not converge
  warnings.warn(


0.7802242476115557

In [13]:
lg_model.score(X_train,y_train)

0.8086337863467076

In [14]:
### predict the X test
y_pred=lg_model.predict_proba(X_test)

In [15]:
## get the probability of the sofwtware to has defect
y_pred_for_defects=y_pred[:,1]

In [16]:
test_data["defects"]=pd.Series(y_pred_for_defects)

In [17]:
submission=test_data[["id","defects"]]

In [18]:
submission.to_csv("submission1.csv",index=False)